In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,date
import warnings
warnings.filterwarnings('ignore')

def preprocess_data_input(data):
    data = data.dropna()
    data = data.drop_duplicates()
    data = data.reset_index()
    data['InvoiceDatetime'] = None
    data['Monetary'] = None
    for i in range (data.shape[0]):
        thoigian = data.loc[i,'InvoiceDate']
        thoigian_1 = datetime.strptime(thoigian,"%d-%m-%Y %H:%M")
        data.loc[i,'InvoiceDatetime'] = thoigian_1
        data.loc[i,'Monetary'] = data.loc[i,'Quantity']*data.loc[i,'UnitPrice']
    data['Monetary'] = data['Monetary'].astype('float64')
    data['InvoiceDatetime'] = pd.to_datetime(data['InvoiceDatetime'])

    data = data[data['UnitPrice'] >= 0]
    
    data_temp = data[data.UnitPrice==0]
    data_temp = set(data_temp.InvoiceNo)
    data_temp = list(data_temp)
    data_temp = data[data['InvoiceNo'].isin(data_temp)]
    data_temp = data_temp.sort_values(by=['InvoiceNo','InvoiceDatetime'])
    data_temp = data_temp.groupby('InvoiceNo')['InvoiceNo'].count()
    data_temp = data_temp.to_frame()
    temp_1 = list(data_temp.index)
    temp_2 = list(data_temp.InvoiceNo)
    data_temp_1 = pd.DataFrame(
        {'InvoiceNo':temp_1,
         'InvoiceNo_ItemNum': temp_2}
    )
    data_temp_1 = data_temp_1[data_temp_1['InvoiceNo_ItemNum']==1]
    data = data[~data['InvoiceNo'].isin(data_temp_1['InvoiceNo'])]

    list_customer = set(data.CustomerID)
    data_pre_process = pd.DataFrame(columns=data_temp.columns)
    for customer in list_customer:
        data_temp = data[data['CustomerID']==customer]
        data_temp = data_temp.sort_values(by=['InvoiceDatetime'], ascending=True)
        list_ind = []
        for ind, val in data_temp.iterrows():
            if val['Quantity'] > 0:
                break
        else:
            list_ind.append(ind)
        if list_ind != []:
            data_temp = data_temp[~data_temp.index.isin(list_ind)]
        else:
            data_temp = data_temp
       
        for stockcode in set(data_temp.StockCode):
            data_temp_1 = data_temp[data_temp['StockCode']==stockcode]
            list_ind_1 = []
            for ind, val in data_temp_1.iterrows():
                if val['Quantity'] > 0:
                    break
                else:
                    list_ind_1.append(ind)
            if list_ind_1 != []:
                data_temp_1 = data_temp_1[~data_temp_1.index.isin(list_ind_1)]
            else:
                data_temp_1 = data_temp_1
            data_temp_2 = data_temp_1[data_temp_1['Quantity']>0]
            data_temp_3 = data_temp_1[data_temp_1['Quantity']<0]
            if data_temp_1['Quantity'].sum()<=0:
                data_pre_process=data_pre_process
            else:
                remain = data_temp_3['Quantity'].sum()
                for j in reversed(data_temp_2.index):
                    remain = data_temp_2.loc[j,"Quantity"] + remain
                    data_temp_2.loc[j,"Quantity"] = remain
                data_temp_2 = data_temp_2[data_temp_2['Quantity']>0]
                data_pre_process = pd.concat([data_pre_process,data_temp_2])    

    data_pre_process['Quantity'] = data_pre_process['Quantity'].astype('int64')
    max_date = date.today()
    Recency = lambda x: (max_date - x.max().date()).days
    Fequency = lambda x: len(x.unique())
    Monetary = lambda x: round(sum(x),2)
    data_rfm = data_pre_process.groupby('CustomerID').agg({
        'InvoiceDatetime':Recency, 
        'InvoiceNo':Fequency, 
        'Monetary':Monetary})
    data_rfm.columns = ['Recency','Frequency','Monetary']
    data_rfm = data_rfm.sort_values('Monetary',ascending=False)

    data_rfm = data_rfm.reset_index(drop = False)
    return data_rfm
    